# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix, classification_report
from sqlalchemy import create_engine
import pickle
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from string import punctuation
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
!ls

categories.csv			   InsertDatabaseName.db
data_cleaned.db			   messages.csv
DisasterResponse.db		   ML Pipeline Preparation.ipynb
ETL Pipeline Preparation.ipynb	   ML Pipeline Preparation-zh.ipynb
ETL Pipeline Preparation-zh.ipynb  Twitter-sentiment-self-drive-DFE.csv


In [2]:
# load data from database
engine = create_engine('sqlite:///data_cleaned.db')
df = pd.read_sql('data_cleaned', engine)
X = df['message'].values
Y = df.drop(['id','message','original','genre'], axis = 1).values

In [4]:
word_tokenize?

In [5]:
df.shape

(26180, 40)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """
    Description:
        This function performs the underlisted process
        on each list of text
        - Removes punctuations
        - Remove stopwords,
        - Normalises the texts
        - Removes whitespaces        
    Params:
        text: list of text to be tokenized
    
    Return:
        clean copy of tokenized text
    """
    # Remove punctuations  
    #text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    text = text.translate(str.maketrans('', '', punctuation))
    
    
    # tokenize text
    tokens = word_tokenize(text)
    
    #instantiantiate lemmmantizer
    lemmatizer = WordNetLemmatizer()

    #lower token, strip whitespaces and remove stopwords
    clean_tokens = [lemmatizer.lemmatize(tok).lower().strip() for tok in tokens]
    return clean_tokens

In [4]:
#print(punctuation)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
CountVectorizer?

In [4]:
pipeline = Pipeline([('count', CountVectorizer(lowercase=False, tokenizer=tokenize, stop_words= 'english')),
                     ('tfidf', TfidfTransformer()),
                         ('clf', MultiOutputClassifier(RandomForestClassifier()))
                        ])

In [5]:
#pipeline?

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.3, random_state = 42) 
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('count', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
      ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
#list(df.drop(['id','message','original','genre'], axis=1).columns)

In [13]:
classification_report?

In [7]:
y_pred = pipeline.predict(X_test)
y_pred_t = np.transpose(y_pred)
y_test_t = np.transpose(y_test)

col_names = list(df.drop(['id','message','original','genre'], axis=1).columns)

i = 0
while i < y_pred.shape[1]:
    print(col_names[i],':\n',classification_report(y_test_t[i], y_pred_t[i]))
    i+=1    

related :
              precision    recall  f1-score   support

          0       0.63      0.47      0.54      1869
          1       0.84      0.91      0.87      5926
          2       0.71      0.08      0.15        59

avg / total       0.79      0.80      0.79      7854

request :
              precision    recall  f1-score   support

          0       0.89      0.98      0.93      6512
          1       0.78      0.41      0.54      1342

avg / total       0.87      0.88      0.86      7854

offer :
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      7816
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      7854

aid_related :
              precision    recall  f1-score   support

          0       0.74      0.86      0.80      4612
          1       0.74      0.57      0.65      3242

avg / total       0.74      0.74      0.74      7854

medical_help :
              precision   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
pipeline.get_params()

{'memory': None,
 'steps': [('count',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=False, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words='english',
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f0e16251400>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_stat

In [16]:
parameters = {'count__ngram_range': [(1,1), (1,2)],
             'tfidf__sublinear_tf': [True, False]
             }

cv = GridSearchCV(pipeline, cv = 5, param_grid=parameters)

In [19]:
cv.fit(X_train,y_train)
cv.best_params_

{'count__ngram_range': (1, 2), 'tfidf__sublinear_tf': False}

In [21]:
TfidfTransformer?

In [8]:
pipeline_rf = Pipeline([
    ('count', CountVectorizer(lowercase=False, tokenizer=tokenize, stop_words= 'english', ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
                         ('clf', MultiOutputClassifier(RandomForestClassifier()))
                        ])

pipeline_rf.fit(X_train, y_train)


y_pred = pipeline_rf.predict(X_test) 

y_pred_t = np.transpose(y_pred)
y_test_t = np.transpose(y_test)

##### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
i = 0
while i < y_pred.shape[1]:
    print('Accuracy:\n {}'.format((y_pred_t[i]==y_test_t[i]).mean()))
    print(col_names[i],':\n',classification_report(y_test_t[i], y_pred_t[i]))
    i+=1   

Accuracy:
 0.7881334351922588
related :
              precision    recall  f1-score   support

          0       0.58      0.51      0.54      1869
          1       0.85      0.88      0.86      5926
          2       0.86      0.10      0.18        59

avg / total       0.78      0.79      0.78      7854

Accuracy:
 0.8758594346829641
request :
              precision    recall  f1-score   support

          0       0.89      0.97      0.93      6512
          1       0.76      0.40      0.53      1342

avg / total       0.87      0.88      0.86      7854

Accuracy:
 0.995161701044054
offer :
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      7816
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      7854

Accuracy:
 0.7379679144385026
aid_related :
              precision    recall  f1-score   support

          0       0.73      0.87      0.80      4612
          1       0.75      0.

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [10]:
pipeline_mn = Pipeline([
    ('count', CountVectorizer(lowercase=False, tokenizer=tokenize, stop_words= 'english', ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
                         ('clf_nb', MultiOutputClassifier(MultinomialNB()))
                        ])

In [11]:
pipeline_mn.fit(X_train, y_train)

y_pred = pipeline_mn.predict(X_test)

y_pred_t = np.transpose(y_pred)


i = 0
while i < y_pred.shape[1]:
    print('Accuracy:\n {}'.format((y_pred_t[i]==y_test_t[i]).mean()))
    print(col_names[i],':\n',classification_report(y_test_t[i], y_pred_t[i]))
    i+=1  

/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


Accuracy:
 0.7636872930990578
related :
              precision    recall  f1-score   support

          0       0.81      0.05      0.09      1869
          1       0.76      1.00      0.86      5926
          2       0.00      0.00      0.00        59

avg / total       0.77      0.76      0.67      7854

Accuracy:
 0.8511586452762924
request :
              precision    recall  f1-score   support

          0       0.85      1.00      0.92      6512
          1       0.95      0.14      0.24      1342

avg / total       0.87      0.85      0.80      7854

Accuracy:
 0.995161701044054
offer :
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      7816
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      7854

Accuracy:
 0.7554112554112554
aid_related :
              precision    recall  f1-score   support

          0       0.75      0.88      0.81      4612
          1       0.77      0.

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
pipeline_kn = Pipeline([
    ('count', CountVectorizer(lowercase=False, tokenizer=tokenize, stop_words= 'english', ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
                         ('clf_kn',MultiOutputClassifier(KNeighborsClassifier()))
                        ]) 

pipeline_kn.fit(X_train, y_train)

y_pred = pipeline_kn.predict(X_test)

y_pred_t = np.transpose(y_pred)


i = 0
while i < y_pred.shape[1]:
    print('Accuracy:\n {}'.format((y_pred_t[i]==y_test_t[i]).mean()))
    print(col_names[i],':\n',classification_report(y_test_t[i], y_pred_t[i]))
    i+=1  

Accuracy:
 0.6354723707664884
related :
              precision    recall  f1-score   support

          0       0.30      0.37      0.33      1869
          1       0.78      0.73      0.75      5926
          2       0.00      0.00      0.00        59

avg / total       0.66      0.64      0.65      7854

Accuracy:
 0.8321874204227145
request :
              precision    recall  f1-score   support

          0       0.83      1.00      0.91      6512
          1       0.75      0.03      0.05      1342

avg / total       0.82      0.83      0.76      7854

Accuracy:
 0.995161701044054
offer :
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      7816
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      7854

Accuracy:
 0.5907817672523555
aid_related :
              precision    recall  f1-score   support

          0       0.59      1.00      0.74      4612
          1       0.71      0.

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [14]:
with open("model.pickle", 'wb') as file:
    pickle.dump(pipeline_rf, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.